In [ ]:
#import geopandas as gpd
import matplotlib.pyplot as plt
#import pyproj
import pandas as pd
import h5py    
import os
from shapely.geometry import Point, Polygon, MultiPolygon
import numpy as np
#import MasterThesis.preprocessing as DP
from shapely.geometry import Polygon
from osgeo import osr, gdal, ogr
import torch
import geopandas as gpd
from MasterThesis import preprocessing as dp
import pickle
import tempfile
from tqdm import tqdm
from MasterThesis import EDA
import os

In [ ]:
path_to_data = "/media/omar/storage/gdrive/Maestria/Datasets" 
metadata  = gpd.read_file(f"{path_to_data}/Frontera_Agricola_Nov2021/Frontera_Agricola_Nov2021.shp")

In [ ]:
metadata.head()

In [ ]:
df = metadata.groupby(["municipio", "elemento"], as_index=True).agg({"area_ha":"mean"}).unstack()
df = df.droplevel(0, axis=1).reset_index().fillna(0)
df.columns = ["Municipality", "non_agricultural_area", "agricultural_area", "other"]
df.head(10)

In [ ]:
def get_folders(path_to_folders):
    
    walk = os.walk(path_to_folders)
    folders = next(walk)[1]
    return folders

In [ ]:
path_to_folders = '/mnt/d/OneDrive - Universidad de Antioquia/sentinel2-colombia-north-west-210GB/01_L1_colombia__2019-01-01__2021-05-30'

In [ ]:
# get Tiles names
tiles_names = get_folders(path_to_folders)
tiles_names.pop(-1)

# Get patches names
patch_names = []
for tile in tiles_names:
    patch_names.append(get_folders(path_to_folders + f"/{tile}"))

    
# Get chip names
chip_names = get_folders(path_to_folders + f'/{tiles_names[0]}' + f'/{patch_names[0][0]}')

#Define path to image
path_to_images = []
patch_list = []
for _tile in tiles_names:
    for patch in patch_names:
        for _patch in patch:
            if _patch[:5] != _tile:
                break
            for _chip in chip_names:
                path_to_images.append(f"{_tile}" + f"/{_patch}" + f"/{_chip}")
                patch_list.append(_patch[:5])

#Save path to images
metadata = pd.DataFrame({"image": path_to_images, "patch": patch_list})
metadata = metadata.drop_duplicates(keep="first")
#metadata.to_csv("metadata_raw_images.csv", index=False)


In [ ]:
metadata

In [ ]:
def get_patch_shapefile(patch_name, path_to_folders, frontera, unique_labels):

    coor = []
    for chip in metadata.query(f"patch=='{patch_name}'").image:

        path_to_chip_metadata = path_to_folders + "/" + chip + "/metadata.pkl"
        corners = pd.read_pickle(path_to_chip_metadata)["corners"]
        nw_x, nw_y = corners["nw"][::-1]
        se_x, se_y = corners["se"][::-1]
        coor.append([nw_x, nw_y, se_x, se_y])

    nw_x = np.array(coor)[:, 0].min()
    nw_y = np.array(coor)[:, 1].max()
    se_x = np.array(coor)[:, 2].max()
    se_y = np.array(coor)[:, 3].min()
    metadata_coor = {"corners": {"nw": np.array([nw_y, nw_x]), "se": np.array([se_y, se_x])}}

    with tempfile.NamedTemporaryFile("wb") as temp:
        pickle.dump(metadata_coor, temp)
        temp.flush()
        patch_reference = patch_name
        gdf_chip = dp.create_shapefiel_from_polygons(temp.name, patch_reference, chip_padding=0.001, crs="epsg:3116")

    gdf_intersection = dp.polygons_intersection(frontera, gdf_chip, unique_labels, patch_reference)
    gdf_intersection.rename({"labels": "elemento"}, axis=1, inplace=True)

    return gdf_intersection

In [ ]:
def create_dataset(metadata, frontera, path_temp_file, path_to_save_masks):

    images = []
    masks = []
    geometry_chip = []
    geometry_intersection = []
    chip_description = []
    current_patch = ""
    unique_labels = frontera["elemento"].unique()
    unique_labels.sort()
    print(unique_labels)
    metadata.sort_values(by="patch", inplace=True)

    for chip_metadata, patch_name in zip(tqdm(metadata.image), metadata.patch):

        chip_reference = chip_metadata.split("/")[1]

        # Create square polygons using chip metada
        gdf_chip = dp.create_shapefiel_from_polygons(
            f"{path_to_folders}/{chip_metadata}/metadata.pkl",
            chip_reference,
            chip_padding=0.0005,
            crs="epsg:3116",
        )

        # get patch shapefile
        if current_patch != patch_name:
            current_patch = patch_name
            patch_shapefile = get_patch_shapefile(patch_name, path_to_folders, frontera, unique_labels)

        # Intersecton between square polygons and SIPRA dataset
        gdf_intersection = dp.polygons_intersection(patch_shapefile, gdf_chip, unique_labels, chip_reference)

        if gdf_intersection.geometry.is_empty.sum() != 3:

            gdf_intersection.to_file(path_temp_file + "/temp.geojson", driver="GeoJSON")

            # create labels masks
            dp.shapefiel_to_geotiff(path_temp_file + "/temp.geojson", f"temp/{chip_reference}.tif", 10, "labels_num", no_data_value=999)

            split_chip_metadata = chip_metadata.split("/")

            if not os.path.isdir(path_to_save_masks + f"/{chip_metadata}"):
                os.makedirs(path_to_save_masks + f"/{chip_metadata}")

            dp.crop_geotiff_chip(
                f"{path_to_folders}/{chip_metadata}/metadata.pkl",
                f"temp/{chip_reference}.tif",
                path_to_save_masks + f"/{chip_metadata}/mask.tif",
            )

            geometry_intersection.append(gdf_intersection)
            masks.append(f"LabelsGeoTiffv2/{chip_metadata}/mask.tif")
            images.append(f"{chip_metadata}/chip.npz")
            metadata = pd.DataFrame({"Image": images, "Mask": masks})

    gdf_intersection = pd.concat(geometry_intersection, ignore_index=True)
    gdf_intersection.crs = "epsg:3116"
    gdf_intersection.to_file("LabelGeotiff/labels.shp")

    return gdf_intersection, metadata

In [ ]:
path_to_data = "/media/omar/storage/gdrive/Maestria/Datasets" 

In [ ]:
path_to_frontera =f"{path_to_data}/Frontera_Agricola_Nov2021/Frontera_Agricola_Nov2021.shp"
frontera = gpd.read_file(path_to_frontera)

In [ ]:
path_to_folders = f"{path_to_data}/Dataset"
path_temp_file = f"{path_to_data}/temp"
path_to_save_masks = f"{path_to_data}/LabelsGeoTiffv-prueba"

In [ ]:
gdf, new_metadata = create_dataset(metadata.query("patch == '18NTP'").iloc[50:60].copy(), frontera, path_temp_file, path_to_save_masks)

In [ ]:
metadata.query("patch == '18NTP'").iloc[50:60].copy()

In [ ]:
{0: "non_agricultural_area",
1: "legal_exclusions",
2: "agricultural_frontier"}

In [ ]:
gdf.plot()

In [ ]:
new_metadata.head()

In [ ]:
old_metadata = pd.read_csv("/mnt/h/Mi unidad/Maestria/Datasets/GeoDataset/metadata.csv")
old_metadata = old_metadata.query("Patch == '18NTP'").iloc[50:60]
old_metadata.head()

In [ ]:
metadata.query("patch == '18NTP'").iloc[50:100].head()

In [ ]:
drive_path = "/mnt/h/Mi unidad"
path_to_label = f"{drive_path}/Maestria/Datasets/"
path_to_images = f"{drive_path}/Maestria/Datasets/GeoDataset/Sentinel_2_Images/"
EDA.visualize_images_and_masks(path_to_label, path_to_images, new_metadata, temporal_dim=False, n=7, figsize=(20, 7))

In [ ]:
drive_path = "/mnt/h/Mi unidad"
path_to_label = f"{drive_path}/Maestria/Datasets/GeoDataset/"
path_to_images = f"{drive_path}/Maestria/Datasets/GeoDataset/Sentinel_2_Images/"
EDA.visualize_images_and_masks(path_to_label, path_to_images, old_metadata, temporal_dim=False, n=7, figsize=(20, 7))

In [ ]:
img_1 = EDA.read_geotiff_image("/mnt/h/Mi unidad/Maestria/Datasets/LabelsGeoTiffv2/18NTN_8_5.tif")
img_2 = EDA.read_geotiff_image("/mnt/h/Mi unidad/Maestria/Datasets/GeoDataset/LabelsGeoTiff/18NTN/18NTN_8_5_(0, 0).tif")

In [ ]:
(img_1 == img_2).sum()

In [ ]:
100*100 - 9970

In [ ]:
plt.imshow(img_1)

In [ ]:
plt.imshow(img_2)

In [ ]:
path_temp_file

In [ ]:
path_temp_file = "/mnt/h/Mi unidad/Maestria/Theses/Preprocessing/temp"

In [ ]:
old_metadata = pd.read_csv("/mnt/h/Mi unidad/Maestria/Datasets/GeoDataset/metadata.csv")
old_metadata

In [ ]:


gdf_intersection.to_file(path_temp_file + '/temp.geojson', driver='GeoJSON')

###create labels masks
DP.shapefiel_to_geotiff(path_temp_file + '/temp.geojson', '/content'+ f'/{chip_reference}.tif', 10, 'labels_num', no_data_value=3)
DP.crop_geotiff_chip(path_to_chip_metadata ,'/content'+ f'/{chip_reference}.tif', path_to_save_masks + f'/{tile}/{chip_reference}.tif')

In [ ]:
clf = gpd.read_file("/mnt/h/Mi unidad/Maestria/Datasets/GeoDataset/metadata_clf.csv")
clf.columns = clf.columns.str.lower()
clf.image = "/" + clf.image.str.replace("/chip.npz", "", regex=False)
gdf = create_dataset(clf.copy())

In [ ]:
gdf.plot()

In [ ]:
clf

In [ ]:
def Create_Dataset(path_to_folders, path_to_frontera, path_to_save_chips, path_to_save_intersection, path_to_save_geotif, path_to_save_masks ,n_tiles=1, n_patches=1, n_chips=1):

    path_temp_file = os.getcwd() + '/temp'
    try:
        os.mkdir(path_temp_file)
    except OSError:
        print("")

    #get Tiles names
    tiles_names = get_folders(path_to_folders)

    #Get patches names
    patch_names = []
    for tile in tiles_names:
        patch_names.append(get_folders(path_to_folders + f'/{tile}'))

    #Get chip names
    chip_names = get_folders(path_to_folders + f'/{tiles_names[0]}' + f'/{patch_names[0][0]}')

    frontera = gpd.read_file(path_to_frontera)

    images = []
    masks = []

    geometry_chip = []
    geometry_intersection = []
    chip_description = []
    for i, tile in enumerate(tiles_names[n_tiles:n_tiles+1], n_tiles):

        try:
            os.mkdir(path_to_save_masks + f"/{tile}")
        except OSError:
            print("")

        for i, patch in enumerate(patch_names[i], 0):#[0:n_patches]:
            print(patch, i)
            for chip in chip_names:#[0:n_chips]:

                chip_reference = f'{patch}' + f'_{chip}'

                exist = os.path.exists(path_to_save_masks + f'/{tile}/{chip_reference}.tif')
                if exist:
                    pass

                else:
                    path_to_chip_metadata = path_to_folders + f'/{tile}' + f'/{patch}' + f'/{chip}' + '/metadata.pkl'

                    #Create square polygons using chip metada
                    gdf_chip = DP.create_shapefiel_from_polygons(path_to_chip_metadata, chip_reference, chip_padding=0.0005, crs='epsg:3116')

                    #Intersecton between square polygons and SIPRA dataset
                    gdf_intersection = DP.polygons_intersection(frontera, gdf_chip, chip_reference)

                    if gdf_intersection.geometry.is_empty.sum() != 3:
                        gdf_intersection.to_file(path_temp_file + '/temp.geojson', driver='GeoJSON')

                        ###create labels masks
                        DP.shapefiel_to_geotiff(path_temp_file + '/temp.geojson', '/content'+ f'/{chip_reference}.tif', 10, 'labels_num', no_data_value=3)
                        DP.crop_geotiff_chip(path_to_chip_metadata ,'/content'+ f'/{chip_reference}.tif', path_to_save_masks + f'/{tile}/{chip_reference}.tif')

                        #Create geotiff from RGB numpy arry
                        #path_to_chip_raster = path_to_folders + f'/{tile}' + f'/{patch}' + f'/{chip}' + '/chip.npz'
                        #chip_raster = np.load(path_to_chip_raster)
                        #chip_rgb_image = chip_raster['arr_0'][0][0:3]
                        #DP.from_array_to_geotiff(path_to_save_geotif + f'/{chip_reference}.tif' , chip_rgb_image, path_to_chip_metadata, crs=3116)


                        #Save square polygons using chip metada
                        #gdf_chip = DP.create_shapefiel_from_polygons(path_to_chip_metadata, chip_reference, crs='epsg:3116')
                        #gdf_intersection = DP.polygons_intersection(frontera, gdf_chip, chip_reference)

                        
                        geometry_intersection.append(gdf_intersection)
                        masks.append(f'LabelsGeoTiff/{tile}/{chip_reference}.tif') 
                        images.append(f'{tile}/{patch}/{chip}/chip.npz') 

                    geometry_chip.append(gdf_chip.geometry.values[0])
                    chip_description.append(chip_reference)
        
            metadata = pd.DataFrame({'Image':images, 'Mask':masks})
            #metadata.to_csv('/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/metadata.csv', index=False, mode = 'a', header=False)
            images = []
            masks = []
    print(tile)
    gdf_chip = gpd.GeoDataFrame({'chip_name':chip_description ,'geometry':geometry_chip}, crs='epsg:3116')
    gdf_intersection = pd.concat(geometry_intersection, ignore_index=True)
    gdf_intersection.crs = 'epsg:3116'

    gdf_chip.to_file(path_to_save_chips)
    gdf_intersection.to_file(path_to_save_intersection)

    return gdf_chip, gdf_intersection, metadata

In [ ]:
path_to_save_chips = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/Chip/18NTN.shp'
path_to_save_intersection = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/Intersection/Intersection.shp'
path_to_save_geotif = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/GeoTiff'
path_to_frontera = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/Frontera_Agricola_Prueba/Frontera_Agricola_Prueba.shp'
path_to_frontera = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/Frontera_Agricola_Nov2021/Frontera_Agricola_Nov2021.shp'
path_to_save_masks = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/LabelsGeoTiff'
path_to_folders = '/content/drive/MyDrive/2021-MAESTRIA-OMAR-CASTAÑO/earth observation datasets/sentinel2-colombia-north-west-290GB/data/colombia/s2_data/01_L1_colombia__2019-01-01__2021-05-30'

In [ ]:
%%time
gdf_chip, gdf_intersection, metadata = Create_Dataset(path_to_folders, path_to_frontera, path_to_save_chips, path_to_save_intersection, path_to_save_geotif,path_to_save_masks, n_chips=1, n_tiles=11, n_patches=1)

In [ ]:
gdf_intersection.head(10)

In [ ]:
gdf_chip.head(10)

In [ ]:
gdf_chip.plot(figsize=(15,7))

In [ ]:
gdf_intersection.plot(figsize=(15,7))

In [ ]:
gdf_intersection.loc[gdf_intersection.labels == "Frontera agrícola nacional"].plot(figsize=(15,7))

In [ ]:
gdf_intersection.loc[gdf_intersection.labels == "Bosques naturales y áreas no agropecuarias"].plot(figsize=(15,7))

In [ ]:
gdf_intersection.loc[gdf_intersection.labels == "Exclusiones legales"].plot(figsize=(15,7))

In [ ]:
#Import libraries
from osgeo import osr, gdal, ogr

In [ ]:
output_raster = path_to_save_masks + "/18NTN/18NTN_8_5_(0, 0).tif"

In [ ]:
output_raster

In [ ]:
 #open geotiff image
raster_image = gdal.Open(output_raster).ReadAsArray()
raster_image.shape

In [ ]:
np.unique(raster_image)

In [ ]:
#plotbinary mask
plt.figure(figsize=(15,8))
plt.imshow(raster_image, cmap='Set1')